# Memoria Práctica 3

Autores: Román García y Patricia Losana

In [3]:
from Datos import Datos
from EstrategiaParticionado import *
from Clasificador import *
#from ClasificadorAG import *
from Roc import *
import numpy as np
from sklearn import preprocessing 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import pprint
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import accuracy_score
from plotModel import plotModel
import matplotlib.pyplot as plt

## 1. Implementación

A continuación se muestran algunos detalles de la implementación llevada a cabo del algoritmo genético.

### 1.1 Generación de la población inicial

Para generar la población inicial es necesario especificar el set de datos a utilizar, y el número de individuos y de generaciones que se desea. 

~~~~
ag = ClasificadorAG(n_cromosomas, dataset, n_generaciones)
~~~~

En ese momento se creará una instancia de la clase ClasificadorAG, donde se generará la población inicial de individuos (cromosomas) con el método generarPoblacion(). 

~~~~
def generarPoblacion(self, n_cromosomas, dataset, regla_entera):
    for _ in range(n_cromosomas):
        self.cromosomas.append(self.Cromosoma(dataset = dataset, regla_entera=regla_entera))
~~~~

Ese método guarda un array de objetos de la clase Cromosoma (donde cada uno de los cuales es un individuo) en el clasificador. Se ha establecido cromosoma como un conjunto de reglas distintas (set()): 
~~~~
class Cromosoma:
    def __init__(self, dataset, reglas = None, regla_entera = True):
        self.regla_entera = regla_entera
        self.n_attrs = len(dataset.nombreAtributos) - 1  #nº attrs menos la clase
        self.rlen = randint(1, pow(dataset.k, self.n_attrs))
        self.datos = dataset.convertirAIntervalos(dataset.datos)
        self.n_intervalos = dataset.k
        self.dataset = dataset
        if reglas != None:
            self.reglas = reglas
        else:
            self.reglas = set()
            self.generarReglas(dataset)

        self.fit = -1
~~~~
El método generarReglas crea objetos

~~~~
class Regla():
    def __init__(self,dataset, regla_entera = True):
        self.regla_entera = regla_entera
        self.valores = []
        self.n_intervalos = dataset.k  #nº attrs menos la clase

        if regla_entera:
            self.valores = np.append(np.random.randint(dataset.k + 1, size = len(dataset.nombreAtributos)-1), np.random.randint(len(dataset.diccionarios['Class']), size = 1))
        else:
            regla = []
            for _ in range(len(dataset.nombreAtributos)-1):
                regla.append(np.random.randint(2, size = dataset.k).tolist())
            regla.append(np.random.randint(len(dataset.diccionarios['Class']), size = 1)[0])
            self.valores = regla
~~~~          
                    
En cuanto al número de reglas se ha establecido un número aleatorio entre 1 y 100todas las posibles combinaciones distintas de intervalos para cada atributo. También se hicieron pruebas con un número aleatorio entre 1 y 20, y los resultados son significativamente mejores en la primera implementación. El principal problema es su elevado coste a nivel temporal.

### 1.2 Mecanismo de cruce implementado

El mecanismo de cruce implementado es entre dos cromosomas por un punto de cruce.



En **clasificador** se llama al recombinar de cromosoma:
~~~~
def recombinar(self):
    poblacion = []
    ruleta = ClasificadorAG.ruleta_rusa(self.cromosomas,len(self.cromosomas))
    for cromosoma in self.cromosomas:
        poblacion.append(cromosoma.recombinar(ruleta.pop(random.randint(0, len (ruleta) - 1))))
    return poblacion
~~~~
Donde el mecanismo de ruleta está basado en:

~~~~
def ruleta_rusa(cromosomas, n_individuos):
    seleccion, probabilidades = [], []

    total_fitness = 0
    for cromosoma in cromosomas:
        total_fitness += cromosoma.fitness()

    if total_fitness == 0:
        return cromosomas
    for cromosoma in cromosomas:
        probabilidades.append(cromosoma.fitness()/total_fitness)
    seleccion = np.random.choice(cromosomas, n_individuos, p=probabilidades)
    return seleccion.tolist()
~~~~

En **cromosoma** se obtiene la probabilidad 
~~~~   
def recombinar(self, cromosoma):
    if tirarDado(99) < 80:
        return self.recombinar_cromosomas(cromosoma)
    return self
~~~~
Donde tirar dado es
~~~~
def tirarDado(caras, ini = 0):
    return random.randint(ini,caras)
~~~~

Y recombinar_cromosomas es:
~~~~
def recombinar_cromosomas(self, cromosoma):
    medio = round(len(self.reglas)/2)
    medio_other = round(len(cromosoma.reglas)/2)

    return ClasificadorAG.Cromosoma(self.dataset, reglas = set(list(self.reglas)[:medio] + list(cromosoma.reglas)[:medio_other]), regla_entera = self.regla_entera )
~~~~


### 1.3 Mecanismo de mutación implementado

El mecanismo de mutación sigue una estrategia parecida al de recombinación: lo que cambia es la probabilidad de ocurrencia. Primero, dentro del **clasificador**
~~~~
def mutar(self):
    poblacion = []
    for cromosoma in self.cromosomas:
        cromosoma.mutar()
~~~~
Después, en **cromosoma**:
~~~~
def mutar(self, porcentaje = 4):
    for regla in self.reglas:
        if tirarDado(99) < porcentaje:
            regla.mutar(porcentaje)
~~~~
Finalmente, a nivel de **regla** se tiene en cuenta si el tipo de regla es binaria o entera:
~~~~
def mutar(self, porcentaje = 4):
    if self.regla_entera:
        for indice, intervalo in enumerate(self.valores[:-1]):
            if tirarDado(99) > porcentaje:
                self.valores[indice] = tirarDado(self.n_intervalos)
    else: 
        for index_attr, attr in enumerate(self.valores[:-1]):
            for index_intervalo, intervalo in enumerate(attr):
                if tirarDado(99) > porcentaje:
                    self.valores[index_attr][index_intervalo] = tirarDado(1)
~~~~

## 2. Resultados de la clasificación

### a) Tamaño de la población = 100; Generaciones = 100

In [11]:
dataset = [Datos("ConjuntosDatos/example1.data"),Datos("ConjuntosDatos/example3.data") , Datos("ConjuntosDatos/example4.data"), Datos("ConjuntosDatos/wdbc.data")]
generaciones = [100, 500, 1000]
poblaciones = [100, 200]

In [ ]:
for data in dataset:  
    for poblacion in poblaciones:
        for generacion in generaciones:
            print("-------------------------------------------------------------------")
            #estrategia = ValidacionCruzada()
            knn = ClasificadorAG(k, normaliza=False)
            val = knn.validacion(estrategia,dataset[data],knn)
            print("Reglas enteras: \n")
            print("Tamaño de la poblacion = ", poblacion, "; \tGeneraciones = ", generaciones)
            print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))

            knn = ClasificadorAG(k, normaliza=False)
            val = knn.validacion(estrategia,dataset[data],knn)
            print("\nReglas binarias: \n")
            print("Tamaño de la poblacion = ", poblacion, "; \tGeneraciones = ", generaciones)
            print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))


In [ ]:
for poblacion in poblaciones:
    for generacion in generaciones:
        print("-------------------------------------------------------------------")
        #estrategia = ValidacionCruzada()
        knn = ClasificadorAG(k, normaliza=False)
        val = knn.validacion(estrategia,dataset[0],knn)

        print("Tamaño de la poblacion = ", poblacion, "; \tGeneraciones = ", generaciones)
        print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))


In [ ]:
for poblacion in poblaciones:
    for generacion in generaciones:
        print("-------------------------------------------------------------------")
        #estrategia = ValidacionCruzada()
        knn = ClasificadorAG(k, normaliza=False)
        val = knn.validacion(estrategia,dataset[0],knn)

        print("Tamaño de la poblacion = ", poblacion, "; \tGeneraciones = ", generaciones)
        print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))


In [ ]:
for poblacion in poblaciones:
    for generacion in generaciones:
        print("-------------------------------------------------------------------")
        #estrategia = ValidacionCruzada()
        knn = ClasificadorAG(k, normaliza=False)
        val = knn.validacion(estrategia,dataset[0],knn)

        print("Tamaño de la poblacion = ", poblacion, "; \tGeneraciones = ", generaciones)
        print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))


### b) Tamaño de la población = 100; Generaciones = 500

### c) Tamaño de la población = 100; Generaciones = 1000

### d) Tamaño de la población = 200; Generaciones = 100

### e) Tamaño de la población = 200; Generaciones = 500

### f) Tamaño de la población = 200; Generaciones = 1000

## 3. Análisis de los resultados

Importancia del número de reglas

Importancia del tamaño de la población

Importancia de las generaciones

Importancia de las tasas de cruce y mutación

Importancia de la representación (enteros o cadenas binarias)

## 4. Representaciones gráficas

### a) Evolución del fitness del mejor individuo

Para cada generación mostrar en pantalla el número de generación y el fitness del mejor individuo

### b) Evolución del fitness medio de la población

Para cada generación mostrar en pantalla el número de generación y el fitness del mejor individuo